In [1]:
import yaml
import torch
import numpy as np
from tqdm import tqdm
from model import SeqLearn
from torch.utils.data import DataLoader
from data import Data

In [2]:
with open("config/bpr.yaml", 'r', encoding='utf-8') as f:
    args = yaml.unsafe_load(f)
args

{'base_path': 'D:/Code/graduation_design/',
 'topk': 10,
 'data': {'device': 'cuda:0',
  'train_test_split': 0.99,
  'base_model_topk': 100,
  'maxlen': 20,
  'name': 'ml-1m',
  'sep': '::',
  'item_path': 'D:/Code/graduation_design/data\\ml-1m\\movies.dat',
  'item_emb_path': 'D:/Code/graduation_design/data\\ml-1m\\item_embeddings.npy',
  'path': 'D:/Code/graduation_design/data\\ml-1m\\ratings.dat',
  'num_negatives': 1,
  'user_threshold': 10,
  'item_threshold': 10,
  'rating_threshold': 2,
  'base_model': ['acf', 'fdsa', 'harnn', 'caser', 'pfmc', 'sasrec', 'anam'],
  'base_model_path': 'D:/Code/graduation_design/base_model_results\\ml-1m'},
 'model': {'lr': 0.001,
  'type': 'SASEM',
  'lamda': 1e-05,
  'hidden_dim': 32,
  'device': 'cuda:0',
  'optimizer': 'AdamOptimizer',
  'tradeoff': 2,
  'div_module': 'cov',
  'pretrain_llm': 'bert-base-uncased',
  'path': 'D:/Code/graduation_design/bpr/ckpt_sigmoid'},
 'epoch': 10,
 'batch_size': 512}

In [3]:
data = Data(args['data'])
train_loader = DataLoader(data.train_dataset, batch_size=args['batch_size'], shuffle=True)

>>>> 数据加载完成: 834449 条交互, 6033 个用户, 3123 个物品
>>>> 基模型的预测结果加载完成: (834449, 7, 102)


>>>> 构建训练集: 100%|██████████| 826104/826104 [00:01<00:00, 438613.06it/s]


>>>> 生成了 826104 个训练样本


>>>> 构建测试集: 100%|██████████| 8345/8345 [00:00<00:00, 835217.67it/s]


>>>> 生成了 8345 个测试样本


In [5]:
model = SeqLearn(args['model'], args['data'], data.n_user, data.n_item)
ckpt = torch.load(f"../bpr/ckpt_score_sum/bpr_epoch9.pth")
filtered_ckpt = {k: v for k, v in ckpt.items() if not k.startswith('item_tower.cex')}
model.load_state_dict(filtered_ckpt, strict=False)
model.eval()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


>>>> 加载预计算的物品嵌入...


SeqLearn(
  (cem): ContentExtractionModule(
    (llm): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 1024, padding_idx=1)
        (position_embeddings): Embedding(514, 1024, padding_idx=1)
        (token_type_embeddings): Embedding(1, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-23): 24 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSdpaSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_features=1024, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dense): Linear

In [11]:
test_loader = DataLoader(data.test_dataset, batch_size=args['batch_size'], shuffle=False)

with torch.no_grad():
    ndcg_scores = []
    for batch in tqdm(test_loader, desc="计算测试集NDCG"):
        user_ids, user_seq, pos_items, neg_items, all_item_scores, base_model_preds = batch

        all_scores = model.predict(user_ids, user_seq, pos_items, neg_items, all_item_scores, base_model_preds)
        scores, indices = torch.topk(all_scores, 10)
        indices += 1

        for i in range(len(user_ids)):
            user_id = user_ids[i].item()
            pos_item = pos_items[i].item()

            true_items = data.user_interacted_items[data.id_to_user[user_id].item()]
            true_items = true_items[true_items.index(pos_items[i]) + 1:]

            predicted_items = np.array([indices[i].cpu().numpy().tolist()])
            ndcg = nDCG(np.array(predicted_items), [true_items])
            ndcg_scores.append(ndcg)

np.mean(ndcg_scores)

计算测试集NDCG: 100%|██████████| 17/17 [01:40<00:00,  5.90s/it]


0.23955300057370915

## 基模型预测

In [77]:
a = torch.tensor([1., 1., 1., 1., 1.])
torch.topk(a, 3)

torch.return_types.topk(
values=tensor([1., 1., 1.]),
indices=tensor([0, 1, 2]))

In [45]:
acf = np.load(args['data']['base_model_path'] + f"/acf.npy")
acf_max = np.max(acf)
acf_min = np.min(acf)
acf_max, acf_min

(6032, 0)

In [18]:
test_loader = DataLoader(data.test_dataset, batch_size=1, shuffle=False)
model = np.load(args['data']['base_model_path'] + f"/sasrec.npy")
print(np.max(predicted_items), np.min(predicted_items))

ndcg_scores = []
phar = tqdm(test_loader, desc="计算NDCG@10...")
for batch in phar:
    user_ids, user_seq, pos_items, neg_items, all_item_scores, base_model_preds = batch

    user_id = user_ids.item()
    pos_item = pos_items.item()
    interaction_idx = data.get_interaction_index(data.id_to_user[user_id], pos_item)
    assert interaction_idx != -1

    predicted_items = model[interaction_idx][2:2+args['topk']]

    predicted_items += 1

    # 获取用户的实际交互物品
    true_items = data.user_interacted_items[user_id]
    true_items = data.user_interacted_items[data.id_to_user[user_id].item()]
    true_items = true_items[true_items.index(pos_item) + 1:]

    ndcg = nDCG(np.array(np.array([predicted_items])), [true_items])
    ndcg_scores.append(ndcg)

    phar.set_postfix(ndcg=ndcg)

np.mean(ndcg_scores)

2241 81


计算NDCG@10...: 100%|██████████| 8345/8345 [00:13<00:00, 596.53it/s, ndcg=[0]]                  


0.13519174826071956

In [ ]:
all_scores = model.predict(user_ids, user_seq, pos_items, base_model_preds)
_, indices = torch.topk(all_scores, 10)
indices

In [ ]:
scores, indices = torch.topk(all_scores, 10)
scores, indices + 1

In [ ]:
true_items = generator.user_interacted_items[generator.id_to_user[user_ids.item()].item()]
len(true_items), true_items[:5], pos_items

In [ ]:
true_items = generator.user_interacted_items[generator.id_to_user[user_ids.item()].item()]
len(true_items), true_items[:5], pos_items

In [ ]:
true_items_clip = true_items[true_items.index(pos_items.item()) + 1:]
len(true_items_clip), true_items_clip[:10]

In [ ]:
x = torch.tensor(2863).unsqueeze(0).to(model.device)
y = torch.tensor(1).unsqueeze(0).to(model.device)

pos_score, neg_score = model(user_ids, user_seq, pos_items, neg_items, base_model_preds)
pos_score, neg_score

In [ ]:
x = torch.tensor(2863).unsqueeze(0).to(model.device)
y = torch.tensor(1).unsqueeze(0).to(model.device)

pos_score, neg_score = model(user_ids, user_seq, pos_items, neg_items, base_model_preds)
pos_score, neg_score

In [ ]:
my_data = DataLoader(train_dataset, batch_size=1, shuffle=True)
user_ids, user_seq, pos_items, neg_items, base_model_preds = next(iter(my_data))
user_ids, user_seq, pos_items, neg_items

In [ ]:
from data import BPRLoss
loss = BPRLoss()

In [10]:
import numpy as np
np.random.seed(2021)
 
class Model:
    def __init__(self, k):
        self.k = k
        self.item_size = 50
 
    def __call__(self, users):
        # 模型随机返回 k 个 item,模拟推荐结果
        res = np.random.randint(0, self.item_size, users.shape[0] * self.k)
        return res.reshape((users.shape[0], -1))
 
 
def get_implict_matrix(rec_items, test_set):
    rel_matrix = [[0] * rec_items.shape[1] for _ in range(rec_items.shape[0])]
    for user in range(len(test_set)):
        for index, item in enumerate(rec_items[user]):
            if item in test_set[user]:
                rel_matrix[user][index] = 1
    return np.array(rel_matrix)
 
 
def DCG(items):
    return np.sum(items / np.log(np.arange(2, len(items) + 2)))
 
 
def nDCG(rec_items, test_set):
    # assert rec_items.shape[0] == len(test_set)
    # 获得隐式反馈的rel分数矩阵
    rel_matrix = get_implict_matrix(rec_items, test_set)
    ndcgs = []
    for user in range(len(test_set)):
        rels = rel_matrix[user]
        dcg = DCG(rels)
        # print(sorted(rels, reverse=True)) # [1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
        idcg = DCG(sorted(rels, reverse=True))
        ndcg = dcg / idcg if idcg != 0 else 0
        ndcgs.append(ndcg)
    return ndcgs
 
 
# 假设 top-20 推荐,一共 5 个 user, 50 个 item ,隐式反馈数据集.
users = np.array([0])
# test_set 表示 5 个用户在测试集中分表交互过那些 item
test_set = [
    [0, 21, 31, 41, 49]
]
rec_items=np.array([
    [0,  9,  5,  6, 7, 50, 8, 31, 21, 1]
])
# model = Model(20)
# rec_items = model(users)
print("truth click", test_set)
print("rec_items", rec_items)
ndcgs = nDCG(rec_items, test_set)
print(ndcgs)
 
print('-'*10)
 
dcg=1/np.log(2)+1/np.log(9)+1/np.log(10)
idcg=1/np.log(2)+1/np.log(3)+1/np.log(4)
ndcg=(1/np.log(2)+1/np.log(9)+1/np.log(10))/(1/np.log(2)+1/np.log(3)+1/np.log(4))
print(dcg, idcg, ndcg)

truth click [[0, 21, 31, 41, 49]]
rec_items [[ 0  9  5  6  7 50  8 31 21  1]]
[0.758586654365518]
----------
2.332109136105634 3.074281787960283 0.758586654365518


In [ ]:
rank_chunk = np.array([
    [[0, 1, 2], [2, 3, 4]],  # 第一个样本的排名结果
    [[1, 2, 3], [3, 4, 0]]   # 第二个样本的排名结果
])

In [ ]:
n_samples, k, topk = rank_chunk.shape  # [batch, k, rank]
rank_chunk_reshape = np.reshape(rank_chunk, [-1, topk])
rank_chunk_reshape

In [ ]:
u_k_i = np.zeros([n_samples * k, 5])
u_k_i

In [ ]:
np.arange(len(u_k_i))

In [ ]:
for i in range(topk):
    u_k_i[np.arange(len(u_k_i)), rank_chunk_reshape[:, i]] = 1 / (i + 10)
u_k_i